In [1]:
import sys
sys.path.append("..")

In [2]:
import numpy as np
import pandas as pd
import time

In [3]:
from meta_qa.integrations import BigQueryIntegration
from meta_qa.tools import run_qa_pipeline
from meta_qa.tools import generate_documentation
from meta_qa.tools import generate_tests_report

In [4]:
integration = BigQueryIntegration("idwall-data", "dw_idwall")

In [5]:
metadata = integration.get_metadata()

/home/danilo.lessa/.local/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [19]:
time1 = time.time()
qa_output = run_qa_pipeline(integration, n_workers=512)
time2 = time.time()

New pandarallel memory created - Size: 2000 MB
Pandarallel will run on 512 workers


In [21]:
delta_t = time2 - time1
tests_data = qa_output["result"]
raw_tests_data = qa_output["raw_result"]
n_ops = raw_tests_data.count().sum()
text = "QA pipeline execution time ({} operations): {:.1f}s".format(n_ops, delta_t)
print(text)

QA pipeline execution time (301 operations): 22.9s


In [22]:
from jinja2 import Environment, FileSystemLoader, select_autoescape
env = Environment(
    loader=FileSystemLoader(searchpath='templates'),
    autoescape=select_autoescape(['html', 'xml'])
)

## Schema

In [23]:
b = metadata.reset_index()


tbl_cols = ["table_name", "table_title", "table_ignore", "table_description_text"]


projects_data  = b.groupby("table_catalog")

projects = []
for project_name, project_data in projects_data:
    
    ds_data = project_data.drop(columns="table_catalog").groupby("table_schema")
    datasets = []
    
    for ds_name, ds_group in ds_data:
        tbl_data = ds_group.drop(columns="table_schema").groupby("table_name")
        tables = []
        for tbl_name, tbl_group in tbl_data:
            
            col_data = tbl_group.drop(columns=tbl_cols)
            tbl_meta = tbl_group.iloc[0][tbl_cols].to_dict()
            tbl_element = col_data.to_dict(orient='records')
            
            headers = []
            columns = []
            for row in tbl_element:
                headers = row.keys()
                columns.append(row.values())
            
            tables.append({**tbl_meta,
                            'headers': headers,
                            'columns': columns})
            
        datasets.append({'dataset_name': ds_name,
                         'tables': tables})
        
    projects.append({'project_name': project_name,
                     'datasets': datasets})

In [ ]:
template = env.get_template('metadata.html')

with open("a.html", "w") as fid:
    fid.write(template.render(projects=projects))

## Tests report

In [ ]:
failed_cells = tests_data.applymap(lambda x: "False" in x).astype(bool)

In [ ]:
success_cells = tests_data.applymap(lambda x: "True" in x).astype(bool)

In [ ]:
failed_rows = failed_cells.sum(axis=1).astype(bool)

In [ ]:
success_rows = success_cells.sum(axis=1).astype(bool)

In [ ]:
failed_cells_2 = tests_data.copy()
failed_cells_2[~failed_cells] = ""
pd.DataFrame(failed_cells_2.apply(lambda x: x.str.cat(), axis=1)).head(5)

In [ ]:
def fail_function(row):
    return " | ".join(row.loc[row].index)

In [ ]:
failed_table = (pd.DataFrame(failed_cells.apply(fail_function, axis=1))
                                         .rename(columns={0: "failed_asserts"})
                                         .where(lambda x: x["failed_asserts"] != "")
                                         .dropna())

In [ ]:
success_table = (pd.DataFrame(success_cells.apply(fail_function, axis=1))
                                         .rename(columns={0: "ok_asserts"})
                                         .where(lambda x: x["ok_asserts"] != "")
                                         .dropna())

In [ ]:
new_table = success_table.join(failed_table).fillna("")

In [ ]:
failed_dataset = failed_table.groupby(["table_schema", "table_name"]).count()

In [ ]:
failed_table.groupby("table_name")

In [ ]:
tables_summary = []



In [ ]:
projects = []
for proj_name, proj_group in new_table.groupby(["table_catalog"]):
    datasets = []
    for ds_name, ds_group in new_table.groupby(["table_schema"]):
        tables = []
        ds_error_count = ds_group.where(lambda x: x["failed_asserts"] != "").count().values[0]
        for table_name, table_group in new_table.groupby(["table_name"]):
            table_summary = table_group.droplevel([0, 1, 2]).reset_index()
            headers = table_summary.columns.tolist()
            columns = table_summary.values.tolist()
            table_name = table_name
            table_error_count = table_group.where(lambda x: x["failed_asserts"] != "").count().values[0]
            tables.append({"table_name": table_name, 
                           "error_count": str(table_error_count),
                           "headers": headers, 
                           "columns": columns})
        datasets.append({"dataset_name": ds_name,
                         "error_count": str(ds_error_count),
                         "tables": tables})
    projects.append({"project_name": proj_name, "datasets": datasets})

In [ ]:
template = env.get_template('metadata.html')
with open("b.html", "w") as fid:
    fid.write(template.render(projects=projects))

In [ ]:
projects[0]["datasets"][0]["dataset_name"]

In [ ]:
projects[0]["datasets"][0]["error_count"]